# İş Problemi
- ID'si verilen kullanıcı için item-based ve user-based recommender
yöntemlerini kullanarak 10 film önerisi yapılacaktır.

# Veri Seti Hikayesi
- Veri seti, bir film tavsiye hizmeti olan MovieLens tarafından sağlanmıştır. İçerisinde filmler ile birlikte bu filmlere yapılan
derecelendirme puanlarını barındırmaktadır. 27.278 filmde 2.000.0263 derecelendirme içermektedir. Bu veri seti ise 17 Ekim 2016
tarihinde oluşturulmuştur. 138.493 kullanıcı ve 09 Ocak 1995 ile 31 Mart 2015 tarihleri arasında verileri içermektedir. Kullanıcılar
rastgele seçilmiştir. Seçilen tüm kullanıcıların en az 20 filme oy verdiği bilgisi mevcuttur.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
movie = pd.read_csv("datasets/movie.csv",low_memory = False)
rating = pd.read_csv("datasets/rating.csv",low_memory = False)

In [3]:
df = movie.merge(rating, how="left", on="movieId")

In [4]:
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


# User Based Recommendation

In [5]:
rating_count = pd.DataFrame(df["title"].value_counts())

In [6]:
rare_movies = rating_count[rating_count["title"] <= 1000].index

In [7]:
rare_movies

Index(['Bear, The (Ours, L') (1988)', 'Rosewood (1997)', 'Ted (2012)',
       'One Night at McCool's (2001)', 'Marked for Death (1990)',
       'Three to Tango (1999)', 'Adam's Rib (1949)',
       'I Now Pronounce You Chuck and Larry (2007)',
       'Italian for Beginners (Italiensk for begyndere) (2000)',
       'Husbands and Wives (1992)',
       ...
       'Satan's Sword (Daibosatsu tôge) (1960)',
       'Blind Massage (Tui na) (2014)', 'Prêt à tout (2014)',
       'Ditchdigger's Daughters, The (1997)', 'A.K. (1985)',
       'Rapture (Arrebato) (1980)',
       'Education of Mohammad Hussein, The (2013)', 'Satanas (2007)',
       'Psychosis (2010)', 'Innocence (2014)'],
      dtype='object', length=24103)

In [8]:
common_movies = df[~df["title"].isin(rare_movies)]

Yaklaşık 3 Milyon değerlendirme veri setinden atıldı.

In [9]:
print(common_movies.shape)
print(df.shape)

(17766015, 6)
(20000797, 6)


In [10]:
user_movie_df = common_movies.pivot_table(index = "userId", columns = "title", values = "rating")

In [11]:
user_movie_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),2046 (2004),21 (2008),21 Grams (2003),21 Jump Street (2012),24 Hour Party People (2002),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Ninjas (1992),3-Iron (Bin-jip) (2004),30 Days of Night (2007),300 (2007),3000 Miles to Graceland (2001),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",42 Up (1998),48 Hrs. (1982),50 First Dates (2004),50/50 (2011),54 (1998),"6th Day, The (2000)","7th Voyage of Sinbad, The (1958)",8 1/2 (8½) (1963),8 Heads in a Duffel Bag (1997),8 Mile (2002),8MM (1999),9 (2009),9 1/2 Weeks (Nine 1/2 Weeks) (1986),"A-Team, The (2010)",A.I. Artificial Intelligence (2001),AVP: Alien vs. Predator (2004),AVPR: Aliens vs. Predator - Requiem (2007),About Last Night... (1986),About Schmidt (2002),About a Boy (2002),"Absent-Minded Professor, The (1961)",Absolute Power (1997),"Abyss, The (1989)",Accepted (2006),"Accidental Tourist, The (1988)","Accused, The (1988)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Across the Universe (2007),Adaptation (2002),Addams Family Values (1993),"Addams Family, The (1991)",Addicted to Love (1997),"Adjustment Bureau, The (2011)",Adventureland (2009),Adventures in Babysitting (1987),"Adventures of Baron Munchausen, The (1988)","Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)","Adventures of Milo and Otis, The (Koneko monogatari) (1986)","Adventures of Pinocchio, The (1996)","Adventures of Priscilla, Queen of the Desert, The (1994)","Adventures of Robin Hood, The (1938)","Adventures of Rocky and Bullwinkle, The (2000)","Adventures of Tintin, The (2011)",Aeon Flux (2005),"Affair to Remember, An (1957)",Affliction (1997),"African Queen, The (1951)",After Hours (1985),"Age of Innocence, The (1993)",Agnes of God (1985),"Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972)",Air America (1990),Air Bud (1997),Air Force One (1997),Airheads (1994),Airplane II: The Sequel (1982),Airplane! (1980),Airport (1970),Akira (1988),Aladdin (1992),Aladdin and the King of Thieves (1996),Alexander (2004),Alfie (2004),Ali (2001),Alice in Wonderland (1951),Alice in Wonderland (2010),Alien (1979),Alien Nation (1988),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Alive (1993),All About Eve (1950),All About My Mother (Todo sobre mi madre) (1999),All Dogs Go to Heaven (1989),All Dogs Go to Heaven 2 (1996),All Quiet on the Western Front (1930),All That Jazz (1979),All the President's Men (1976),All the Pretty Horses (2000),Almost Famous (2000),Along Came Polly (2004),Along Came a Spider (2001),"Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965)",Altered States (1980),Always (1989),Amadeus (1984),Amateur (1994),"Amazing Spider-Man, The (2012)","Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",America's Sweethearts (2001),American Beauty (1999),American Gangster (2007),American Gigolo (1980),American Graffiti (1973),American History X (1998),American Hustle (2013),American Movie (1999),American Pie (1999),American Pie 2 (2001),"American President, The (1995)",American Psycho (2000),American Splendor (2003),"American Tail, An (1986)","American Tail: Fievel Goes West, An (1991)",Amer

user_movie_df'in oluşturma aşamalarının fonksiyonlaştırılması

In [12]:
def create_user_movie_df(df, rare_movies_count = 1000):
    rating_count = pd.DataFrame(df["title"].value_counts())
    rare_movies = rating_count[rating_count["title"] <= rare_movies_count].index
    common_movies = df[~df["title"].isin(rare_movies)]
    return common_movies.pivot_table(index = "userId", columns = "title", values = "rating")


Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi

In [13]:
def create_random_user(user_movie_df):
    return int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)

In [14]:
random_user = create_random_user(user_movie_df)

In [15]:
random_user

28941

In [16]:
random_user_df = user_movie_df[user_movie_df.index == random_user]

In [17]:
random_user_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),2046 (2004),21 (2008),21 Grams (2003),21 Jump Street (2012),24 Hour Party People (2002),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Ninjas (1992),3-Iron (Bin-jip) (2004),30 Days of Night (2007),300 (2007),3000 Miles to Graceland (2001),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",42 Up (1998),48 Hrs. (1982),50 First Dates (2004),50/50 (2011),54 (1998),"6th Day, The (2000)","7th Voyage of Sinbad, The (1958)",8 1/2 (8½) (1963),8 Heads in a Duffel Bag (1997),8 Mile (2002),8MM (1999),9 (2009),9 1/2 Weeks (Nine 1/2 Weeks) (1986),"A-Team, The (2010)",A.I. Artificial Intelligence (2001),AVP: Alien vs. Predator (2004),AVPR: Aliens vs. Predator - Requiem (2007),About Last Night... (1986),About Schmidt (2002),About a Boy (2002),"Absent-Minded Professor, The (1961)",Absolute Power (1997),"Abyss, The (1989)",Accepted (2006),"Accidental Tourist, The (1988)","Accused, The (1988)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Across the Universe (2007),Adaptation (2002),Addams Family Values (1993),"Addams Family, The (1991)",Addicted to Love (1997),"Adjustment Bureau, The (2011)",Adventureland (2009),Adventures in Babysitting (1987),"Adventures of Baron Munchausen, The (1988)","Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)","Adventures of Milo and Otis, The (Koneko monogatari) (1986)","Adventures of Pinocchio, The (1996)","Adventures of Priscilla, Queen of the Desert, The (1994)","Adventures of Robin Hood, The (1938)","Adventures of Rocky and Bullwinkle, The (2000)","Adventures of Tintin, The (2011)",Aeon Flux (2005),"Affair to Remember, An (1957)",Affliction (1997),"African Queen, The (1951)",After Hours (1985),"Age of Innocence, The (1993)",Agnes of God (1985),"Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972)",Air America (1990),Air Bud (1997),Air Force One (1997),Airheads (1994),Airplane II: The Sequel (1982),Airplane! (1980),Airport (1970),Akira (1988),Aladdin (1992),Aladdin and the King of Thieves (1996),Alexander (2004),Alfie (2004),Ali (2001),Alice in Wonderland (1951),Alice in Wonderland (2010),Alien (1979),Alien Nation (1988),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Alive (1993),All About Eve (1950),All About My Mother (Todo sobre mi madre) (1999),All Dogs Go to Heaven (1989),All Dogs Go to Heaven 2 (1996),All Quiet on the Western Front (1930),All That Jazz (1979),All the President's Men (1976),All the Pretty Horses (2000),Almost Famous (2000),Along Came Polly (2004),Along Came a Spider (2001),"Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965)",Altered States (1980),Always (1989),Amadeus (1984),Amateur (1994),"Amazing Spider-Man, The (2012)","Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",America's Sweethearts (2001),American Beauty (1999),American Gangster (2007),American Gigolo (1980),American Graffiti (1973),American History X (1998),American Hustle (2013),American Movie (1999),American Pie (1999),American Pie 2 (2001),"American President, The (1995)",American Psycho (2000),American Splendor (2003),"American Tail, An (1986)","American Tail: Fievel Goes West, An (1991)",Amer

In [18]:
movies_watched = random_user_df.columns[random_user_df.notna().any()].to_list()

In [19]:
len(movies_watched)

33

Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişilmesi

In [20]:
movies_watched_df = user_movie_df[movies_watched]

In [21]:
movies_watched_df

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),Four Weddings and a Funeral (1994),Home Alone (1990),Jurassic Park (1993),Like Water for Chocolate (Como agua para chocolate) (1992),Little Women (1994),Mr. Holland's Opus (1995),Mrs. Doubtfire (1993),Much Ado About Nothing (1993),Muriel's Wedding (1994),Nine Months (1995),Operation Dumbo Drop (1995),"Piano, The (1993)","Postman, The (Postino, Il) (1994)",Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0
4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN
5.0,NaN,NaN,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,3.0,NaN,3.0,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
138490.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,5.0,NaN,NaN
138491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
user_movie_count = movies_watched_df.T.notnull().sum()

In [23]:
user_movie_count.head(2)

userId
1.0    1
2.0    2
dtype: int64

In [24]:
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]

In [25]:
user_movie_count.head(2)

,userId,movie_count
0,1.0,1
1,2.0,2


In [26]:
perc = len(movies_watched) * 60 / 100

In [27]:
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]

In [28]:
users_same_movies

90            91.0
129          130.0
155          156.0
157          158.0
159          160.0
            ...   
138207    138208.0
138278    138279.0
138381    138382.0
138414    138415.0
138482    138483.0
Name: userId, Length: 4139, dtype: float64

Öneri Yapılacak Kullanıcı ile En Benzer Kullanıcıların Belirlenmesi

In [29]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                     random_user_df[movies_watched]])

In [30]:
final_df

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),Four Weddings and a Funeral (1994),Home Alone (1990),Jurassic Park (1993),Like Water for Chocolate (Como agua para chocolate) (1992),Little Women (1994),Mr. Holland's Opus (1995),Mrs. Doubtfire (1993),Much Ado About Nothing (1993),Muriel's Wedding (1994),Nine Months (1995),Operation Dumbo Drop (1995),"Piano, The (1993)","Postman, The (Postino, Il) (1994)",Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
91.0,2.5,2.0,NaN,4.0,3.5,NaN,NaN,NaN,NaN,4.0,4.5,3.0,3.5,NaN,3.5,3.5,4.0,4.0,3.5,2.5,2.5,NaN,NaN,NaN,NaN,2.5,4.5,NaN,NaN,NaN,3.0,3.5,4.0
130.0,4.0,3.0,NaN,3.0,3.0,NaN,NaN,3.0,5.0,5.0,5.0,3.0,4.0,3.0,NaN,4.0,4.0,NaN,3.0,NaN,NaN,4.0,4.0,NaN,3.0,NaN,5.0,NaN,NaN,3.0,5.0,NaN,3.0
156.0,3.0,NaN,NaN,5.0,5.0,3.0,NaN,NaN,4.0,5.0,3.0,4.0,5.0,NaN,NaN,4.0,5.0,4.0,3.0,3.0,4.0,5.0,NaN,NaN,NaN,4.0,5.0,NaN,4.0,4.0,5.0,3.0,4.0
158.0,2.0,1.0,4.0,4.0,3.0,5.0,NaN,4.0,NaN,5.0,4.0,3.0,3.0,NaN,NaN,3.0,3.0,NaN,5.0,3.0,3.0,NaN,5.0,NaN,5.0,3.0,5.0,5.0,4.0,5.0,5.0,NaN,NaN
160.0,NaN,NaN,3.0,5.0,4.0,4.0,NaN,3.0,2.0,5.0,5.0,4.0,4.0,NaN,4.0,NaN,4.0,3.0,NaN,3.0,NaN,3.0,NaN,NaN,5.0,NaN,5.0,NaN,NaN,5.0,4.0,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138279.0,3.0,NaN,NaN,3.0,5.0,5.0,5.0,4.0,NaN,5.0,4.0,2.0,5.0,5.0,5.0,4.0,3.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,5.0,NaN,4.0,NaN,5.0,3.0,3.0
138382.0,1.0,1.0,4.0,2.0,3.0,5.0,NaN,4.0,3.0,5.0,3.0,3.0,5.0,5.0,NaN,NaN,5.0,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,3.0,NaN,3.0,3.0,NaN,5.0,NaN,4.0
138415.0,1.0,NaN,3.0,4.0,3.0,5.0,NaN,NaN,NaN,4.0,5.0,NaN,3.0,3.0,3.0,4.0,4.0,NaN,4.0,4.0,NaN,3.0,5.0,NaN,5.0,NaN,5.0,3.0,4.0,4.0,NaN,3.0,NaN


In [31]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()

In [32]:
corr_df

userId    userId  
105664.0  78262.0    -0.954490
119715.0  55005.0    -0.952579
104652.0  137558.0   -0.950000
72596.0   29398.0    -0.939060
28866.0   67756.0    -0.936065
                        ...   
69856.0   15366.0     0.978167
111108.0  117465.0    1.000000
28941.0   28941.0     1.000000
16562.0   41005.0     1.000000
91.0      16562.0          NaN
Length: 5556793, dtype: float64

In [33]:
corr_df = pd.DataFrame(corr_df, columns = ["corr"])
corr_df.index.names = ["userId1","userId2"]

In [34]:
corr_df.head()

,,corr
userId1,userId2,
105664.0,78262.0,-0.954490
119715.0,55005.0,-0.952579
104652.0,137558.0,-0.950000
72596.0,29398.0,-0.939060
28866.0,67756.0,-0.936065


In [35]:
corr_df.reset_index(inplace = True)

In [36]:
top_users = corr_df[(corr_df["userId1"] == random_user) & (corr_df["corr"] >= 0.65)][["userId2", "corr"]].reset_index(drop = True)

In [37]:
top_users.head()

,userId2,corr
0,105474.0,0.651830
1,105474.0,0.651830
2,82666.0,0.655212
3,82666.0,0.655212
4,94379.0,0.655596


In [38]:
top_users.rename(columns = {"userId2":"userId"}, inplace = True)

In [39]:
top_users.head()

,userId,corr
0,105474.0,0.651830
1,105474.0,0.651830
2,82666.0,0.655212
3,82666.0,0.655212
4,94379.0,0.655596


In [40]:
top_users_rating = top_users.merge(rating[["userId","movieId","rating"]], how = "inner")

In [41]:
top_users_rating.head()

,userId,corr,movieId,rating
0,105474.0,0.65183,1,4.0
1,105474.0,0.65183,3,3.0
2,105474.0,0.65183,5,3.0
3,105474.0,0.65183,7,4.0
4,105474.0,0.65183,11,3.0


In [42]:
top_users_rating = top_users_rating[top_users_rating["userId"] != random_user]

Weighted Average Recommendation Score'un Hesaplanması ve İlk 5 Filmin Tutulması

In [43]:
top_users_rating["weighted_rating"] = top_users_rating["corr"] * top_users_rating["rating"]

In [44]:
top_users_rating.head(2)

,userId,corr,movieId,rating,weighted_rating
0,105474.0,0.65183,1,4.0,2.60732
1,105474.0,0.65183,3,3.0,1.95549


In [45]:
recommendation_df = top_users_rating.groupby("movieId").agg({"weighted_rating":"mean"})

In [46]:
recommendation_df.head()

,weighted_rating
movieId,
1,2.354019
2,1.511405
3,1.763924
4,1.705451
5,1.535460


In [47]:
recommendation_df.reset_index(inplace=True)

In [48]:
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending = False)

In [49]:
movies_to_be_recommend = movies_to_be_recommend.merge(movie[["movieId", "title"]])

In [50]:
movies_to_be_recommend.head()

,movieId,weighted_rating,title
0,1922,3.76358,Whatever (1998)
1,2057,3.76358,"Incredible Journey, The (1963)"
2,2077,3.76358,"Journey of Natty Gann, The (1985)"
3,2485,3.76358,She's All That (1999)
4,2577,3.76358,Metroland (1997)


User Based Recommendation Fonksiyonlaştırılması

In [51]:
def create_movies_to_be_recommend(random_user, user_movie_df, movie, rating, ratio = 60, cor_th = 0.65, score = 3.5):
    random_user_df = user_movie_df[user_movie_df.index == random_user]
    movies_watched = random_user_df.columns[random_user_df.notna().any()].to_list()
    movies_watched_df = user_movie_df[movies_watched]

    user_movie_count = movies_watched_df.T.notnull().sum()
    user_movie_count = user_movie_count.reset_index()
    user_movie_count.columns = ["userId", "movie_count"]

    perc = len(movies_watched) * ratio / 100
    users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]

    final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                     random_user_df[movies_watched]])
    
    corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
    corr_df = pd.DataFrame(corr_df, columns = ["corr"])
    corr_df.index.names = ["userId1","userId2"]
    corr_df.reset_index(inplace = True)

    top_users = corr_df[(corr_df["userId1"] == random_user) & (corr_df["corr"] >= cor_th)][["userId2", "corr"]].reset_index(drop = True)
    top_users.rename(columns = {"userId2":"userId"}, inplace = True)
    top_users_rating = top_users.merge(rating[["userId","movieId","rating"]], how = "inner")
    top_users_rating = top_users_rating[top_users_rating["userId"] != random_user]

    top_users_rating["weighted_rating"] = top_users_rating["corr"] * top_users_rating["rating"]

    recommendation_df = top_users_rating.groupby("movieId").agg({"weighted_rating":"mean"})
    recommendation_df.reset_index(inplace=True)

    movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > score].sort_values("weighted_rating", ascending = False)
    movies_to_be_recommend = movies_to_be_recommend.merge(movie[["movieId", "title"]])

    return movies_to_be_recommend

In [52]:
movies_to_be_recommend_xx = create_movies_to_be_recommend(random_user, user_movie_df, movie, rating)

In [53]:
movies_to_be_recommend_xx.head()

,movieId,weighted_rating,title
0,1922,3.76358,Whatever (1998)
1,2057,3.76358,"Incredible Journey, The (1963)"
2,2077,3.76358,"Journey of Natty Gann, The (1985)"
3,2485,3.76358,She's All That (1999)
4,2577,3.76358,Metroland (1997)


# Item Based Recommendation

In [54]:
movie_id = rating[(rating["userId"] == random_user) & (rating["rating"] == 5)].sort_values(by = "timestamp", ascending = False)["movieId"].iloc[0:1].values[0]

In [55]:
movie_name = movie[movie["movieId"] == movie_id]["title"].values[0]

In [56]:
user_movie_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),2046 (2004),21 (2008),21 Grams (2003),21 Jump Street (2012),24 Hour Party People (2002),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Ninjas (1992),3-Iron (Bin-jip) (2004),30 Days of Night (2007),300 (2007),3000 Miles to Graceland (2001),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",42 Up (1998),48 Hrs. (1982),50 First Dates (2004),50/50 (2011),54 (1998),"6th Day, The (2000)","7th Voyage of Sinbad, The (1958)",8 1/2 (8½) (1963),8 Heads in a Duffel Bag (1997),8 Mile (2002),8MM (1999),9 (2009),9 1/2 Weeks (Nine 1/2 Weeks) (1986),"A-Team, The (2010)",A.I. Artificial Intelligence (2001),AVP: Alien vs. Predator (2004),AVPR: Aliens vs. Predator - Requiem (2007),About Last Night... (1986),About Schmidt (2002),About a Boy (2002),"Absent-Minded Professor, The (1961)",Absolute Power (1997),"Abyss, The (1989)",Accepted (2006),"Accidental Tourist, The (1988)","Accused, The (1988)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Across the Universe (2007),Adaptation (2002),Addams Family Values (1993),"Addams Family, The (1991)",Addicted to Love (1997),"Adjustment Bureau, The (2011)",Adventureland (2009),Adventures in Babysitting (1987),"Adventures of Baron Munchausen, The (1988)","Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)","Adventures of Milo and Otis, The (Koneko monogatari) (1986)","Adventures of Pinocchio, The (1996)","Adventures of Priscilla, Queen of the Desert, The (1994)","Adventures of Robin Hood, The (1938)","Adventures of Rocky and Bullwinkle, The (2000)","Adventures of Tintin, The (2011)",Aeon Flux (2005),"Affair to Remember, An (1957)",Affliction (1997),"African Queen, The (1951)",After Hours (1985),"Age of Innocence, The (1993)",Agnes of God (1985),"Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972)",Air America (1990),Air Bud (1997),Air Force One (1997),Airheads (1994),Airplane II: The Sequel (1982),Airplane! (1980),Airport (1970),Akira (1988),Aladdin (1992),Aladdin and the King of Thieves (1996),Alexander (2004),Alfie (2004),Ali (2001),Alice in Wonderland (1951),Alice in Wonderland (2010),Alien (1979),Alien Nation (1988),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Alive (1993),All About Eve (1950),All About My Mother (Todo sobre mi madre) (1999),All Dogs Go to Heaven (1989),All Dogs Go to Heaven 2 (1996),All Quiet on the Western Front (1930),All That Jazz (1979),All the President's Men (1976),All the Pretty Horses (2000),Almost Famous (2000),Along Came Polly (2004),Along Came a Spider (2001),"Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965)",Altered States (1980),Always (1989),Amadeus (1984),Amateur (1994),"Amazing Spider-Man, The (2012)","Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",America's Sweethearts (2001),American Beauty (1999),American Gangster (2007),American Gigolo (1980),American Graffiti (1973),American History X (1998),American Hustle (2013),American Movie (1999),American Pie (1999),American Pie 2 (2001),"American President, The (1995)",American Psycho (2000),American Splendor (2003),"American Tail, An (1986)","American Tail: Fievel Goes West, An (1991)",Amer

In [57]:
my_movie_df = user_movie_df[movie_name]

In [58]:
my_movie_df

userId
1.0        NaN
2.0        NaN
3.0        NaN
4.0        NaN
5.0        NaN
            ..
138489.0   NaN
138490.0   NaN
138491.0   NaN
138492.0   NaN
138493.0   NaN
Name: Sabrina (1995), Length: 138493, dtype: float64

In [59]:
item_based_recommender_df = user_movie_df.corrwith(my_movie_df).sort_values(ascending = False)

In [60]:
item_based_recommender_df.head()

Sabrina (1995)                1.000000
Intouchables (2011)           0.503128
Father of the Bride (1991)    0.500332
Anna and the King (1999)      0.494133
Runaway Bride (1999)          0.483923
dtype: float64

In [61]:
item_based_recommender_df[1:6]

Intouchables (2011)                 0.503128
Father of the Bride (1991)          0.500332
Anna and the King (1999)            0.494133
Runaway Bride (1999)                0.483923
Phantom of the Opera, The (2004)    0.476132
dtype: float64

Çalışmanın Basit bir Scriptini Hazırlayalım

In [62]:
def create_item_based_recommender(movie_name, user_movie_df):
    my_movie_df = user_movie_df[movie_name]
    item_based_recommender_df = user_movie_df.corrwith(my_movie_df).sort_values(ascending = False)
    return item_based_recommender_df[1:]


In [63]:
item_based_recommender2 = create_item_based_recommender(movie_name, user_movie_df)

In [64]:
item_based_recommender2.head()

Intouchables (2011)                 0.503128
Father of the Bride (1991)          0.500332
Anna and the King (1999)            0.494133
Runaway Bride (1999)                0.483923
Phantom of the Opera, The (2004)    0.476132
dtype: float64